In [1]:
from functions import transform_functions as traf
from functions import preprossesing as prep
from functions import folding
import copy
import sys
import numpy as np
import pandas as pd
import tensorflow as tf
from six import unichr
from operator import itemgetter
from datetime import datetime
from collections import Counter
from tensorflow.python.keras.models import Model
from tensorflow.python.keras.layers.core import Dense
from tensorflow.python.keras.layers import LSTM, Input
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
import csv
from numpy.compat import unicode


filename = "running-example"
ocel, act_dict, cust_dict = traf.prepare_ocel(filename)
col = ["Items", "Products", "Orders",]
# for c in col:
#     print(c)
#     print(np.unique([len(t) for t in ocel[c]]))
print(act_dict)
print(cust_dict)
# display(ocel.head())
drops_col_order = ["weight", "price", "Event_ID", 'Products']
ocel_orders = traf.flatten(ocel, 'Orders')
enriched_log, single_log = traf.gen_flatted_comp_csv(
    ocel_orders, 'Orders', drops_col_order)


{'place order': 8, 'pick item': 7, 'confirm order': 0, 'item out of stock': 3, 'reorder item': 9, 'pay order': 5, 'create package': 1, 'send package': 10, 'failed delivery': 2, 'package delivered': 4, 'payment reminder': 6}
{'Marco Pegoraro': 12, 'Gyunam Park': 4, 'Majid Rafiei': 11, 'Junxiong Gao': 5, 'Seran Uysal': 14, 'Christina Rensinghof': 1, 'Wil van der Aalst': 16, 'Christine Dobbert': 2, 'Luis Santos': 8, 'Kefang Ding': 6, 'Mohammadreza Fani Sani': 13, 'Tobias Brockhoff': 15, 'Anahita Farhang Ghahfarokhi': 0, 'Mahnaz Qafari': 9, 'Claudia Graf': 3, 'Mahsa Bafrani': 10, 'Lisa Mannel': 7}


In [2]:
len(enriched_log['Case_ID'].unique())

2000

In [3]:
np.random.seed(42)

lastcase = ''
line = ''
firstLine = True
lines = []
Ptimeseqs = []
Ptimeseqs2 = []
Ptimeseqs3 = []
Ptimeseqs4 = []

Ptimes = []
Ptimes2 = []
Ptimes3 = []
Ptimes4 = []
ascii_offset = 64
numlines = 0
Pcasestarttime = None
Plasteventtime = None

for index, row in enriched_log.iterrows():
    t = row['Timestamp']

    if row['Case_ID'] != lastcase:
        Stime = t
        Pcasestarttime = t
        Plasteventtime = t
        lastcase = row['Case_ID']
        if not firstLine:
            lines.append(line)
            Ptimeseqs.append(Ptimes)
            Ptimeseqs2.append(Ptimes2)
            Ptimeseqs3.append(Ptimes3)
            Ptimeseqs4.append(Ptimes4)

        line = ''
        Ptimes = []
        Ptimes2 = []
        Ptimes3 = []
        Ptimes4 = []

        numlines += 1

    line += unichr(int(row['Activity']) + ascii_offset)
    Ptimesincelastevent = t - Plasteventtime
    Ptimesincecasestart = t - Pcasestarttime
    midnight = t.replace(hour=0, minute=0, second=0, microsecond=0)
    timesincemidnight = t - midnight
    Ptimediff = 86400 * Ptimesincelastevent.days + Ptimesincelastevent.seconds
    Ptimediff2 = 86400 * Ptimesincecasestart.days + Ptimesincecasestart.seconds
    Ptimediff3 = timesincemidnight.seconds
    Ptimediff4 = t.weekday()
    Ptimes.append(Ptimediff)
    Ptimes2.append(Ptimediff2)
    Ptimes3.append(Ptimediff3)
    Ptimes4.append(Ptimediff4)

    Plasteventtime = t
    firstLine = False

# Add last case
lines.append(line)
Ptimeseqs.append(Ptimes)
Ptimeseqs2.append(Ptimes2)
Ptimeseqs3.append(Ptimes3)
Ptimeseqs4.append(Ptimes4)

PtimeseqsF = []
for seq in Ptimeseqs2:
    PtimeseqsF.append([seq[-1] - t for t in seq])

numlines += 1


divisor = np.mean([item for sublist in Ptimeseqs for item in sublist])  # average time between events
divisor2 = np.mean([item for sublist in Ptimeseqs2 for item in sublist])  # average time between current and first events
divisorTR = np.mean([item for sublist in PtimeseqsF for item in sublist])  # average time instance remaining
print(f"divisor: {divisor}")
print(f"divisor2: {divisor2}")
print(f"divisorTR: {divisorTR}")

### folding the lists 
lines, lines_t, lines_t2, lines_t3, lines_t4, lines_t5 = folding.folding_this(lines,Ptimeseqs,Ptimeseqs2,Ptimeseqs3,Ptimeseqs4,PtimeseqsF,numlines,seeded = 42)

step = 1
sentences = []
softness = 0
next_chars = []
lines = list(map(lambda x: x + '!', lines))  # put delimiter symbol
maxlen = max(map(lambda x: len(x), lines))  # find maximum line size

# next lines here to get all possible characters for events and annotate them with numbers
# chars = map(lambda x: set(x),lines)
chars = list(set().union(*map(lambda x: set(x), lines)))
chars.sort()
target_chars = copy.copy(chars)
chars.remove('!')
print('total chars: {}, target chars: {}'.format(len(chars), len(target_chars)))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))
target_char_indices = dict((c, i) for i, c in enumerate(target_chars))
target_indices_char = dict((i, c) for i, c in enumerate(target_chars))
print(indices_char)
sentences_t = []
sentences_t2 = []
sentences_t3 = []
sentences_t4 = []
sentences_t5 = []

next_chars_t = []
next_chars_t2 = []
next_chars_t3 = []
next_chars_t4 = []
next_chars_t5 = []

for line, line_t, line_t2, line_t3, line_t4, line_t5 in zip(lines, lines_t, lines_t2, lines_t3, lines_t4, lines_t5):
    for i in range(0, len(line), step):
        if i == 0:
            continue

        # we add iteratively, first symbol of the line, then two first, three...

        sentences.append(line[0: i])
        sentences_t.append(line_t[0:i])
        sentences_t2.append(line_t2[0:i])
        sentences_t3.append(line_t3[0:i])
        sentences_t4.append(line_t4[0:i])
        sentences_t5.append(line_t5[0:i])

        next_chars.append(line[i])
        if i == len(line) - 1:  # special case to deal time of end character
            next_chars_t.append(0)
            next_chars_t2.append(0)
            next_chars_t3.append(0)
            next_chars_t4.append(0)
            next_chars_t5.append(0)


        else:
            next_chars_t.append(line_t[i])
            next_chars_t2.append(line_t2[i])
            next_chars_t3.append(line_t3[i])
            next_chars_t4.append(line_t4[i])
            next_chars_t5.append(line_t5[i])

print('nb sequences:', len(sentences))
print('Vectorization...')
num_features = len(chars) + 5
print('num features: {}'.format(num_features))
X = np.zeros((len(sentences), maxlen, num_features), dtype=np.float32)
y_a = np.zeros((len(sentences), len(target_chars)), dtype=np.float32)
y_t = np.zeros((len(sentences)), dtype=np.float32)
y_tr = np.zeros((len(sentences)), dtype=np.float32)
for i, sentence in enumerate(sentences):
    leftpad = maxlen - len(sentence)
    next_t = next_chars_t[i]
    next_tr = next_chars_t5[i]

    sentence_t = sentences_t[i]
    sentence_t2 = sentences_t2[i]
    sentence_t3 = sentences_t3[i]
    sentence_t4 = sentences_t4[i]

    for t, char in enumerate(sentence):
        multiset_abstraction = Counter(sentence[:t + 1])
        for c in chars:
            if c == char:  # this will encode present events to the right places
                X[i, t + leftpad, char_indices[c]] = 1
        X[i, t + leftpad, len(chars)] = t + 1
        X[i, t + leftpad, len(chars) + 1] = sentence_t[t] / divisor
        X[i, t + leftpad, len(chars) + 2] = sentence_t2[t] / divisor2
        X[i, t + leftpad, len(chars) + 3] = sentence_t3[t] / 86400
        X[i, t + leftpad, len(chars) + 4] = sentence_t4[t] / 7

    for c in target_chars:
        if c == next_chars[i]:
            y_a[i, target_char_indices[c]] = 1 - softness
        else:
            y_a[i, target_char_indices[c]] = softness / (len(target_chars) - 1)
    y_t[i] = next_t / divisor
    y_tr[i] = next_tr / divisorTR

    np.set_printoptions(threshold=sys.maxsize)


divisor: 103436.38622985176
divisor2: 604261.1036767652
divisor2: 1248093.355995932
total chars: 11, target chars: 12
{0: '@', 1: 'A', 2: 'B', 3: 'C', 4: 'D', 5: 'E', 6: 'F', 7: 'G', 8: 'H', 9: 'I', 10: 'J'}
nb sequences: 21620
Vectorization...
num features: 16


In [40]:
print(len(sentences))

21620


In [ ]:
print('Build model...')
main_input = Input(shape=(maxlen, num_features), name='main_input')
# train a 2-layer LSTM with one shared layer
l1 = LSTM(100, implementation=2, kernel_initializer='glorot_uniform', return_sequences=True, dropout=0.2)(
    main_input)  # the shared layer
b1 = tf.keras.layers.BatchNormalization()(l1)
l2_1 = LSTM(100, implementation=2, kernel_initializer='glorot_uniform', return_sequences=False, dropout=0.2)(
    b1)  # the layer specialized in activity prediction
b2_1 = tf.keras.layers.BatchNormalization()(l2_1)
l2_2 = LSTM(100, implementation=2, kernel_initializer='glorot_uniform', return_sequences=False, dropout=0.2)(
    b1)  # the layer specialized in time prediction
b2_2 = tf.keras.layers.BatchNormalization()(l2_2)
l2_3 = LSTM(100, implementation=2, kernel_initializer='glorot_uniform', return_sequences=False, dropout=0.2)(
    b1)  # the layer specialized in time remaining prediction
b2_3 = tf.keras.layers.BatchNormalization()(l2_3)
act_output = Dense(len(target_chars), activation='softmax', kernel_initializer='glorot_uniform', name='act_output')(
    b2_1)
time_output = Dense(1, kernel_initializer='glorot_uniform', name='time_output')(b2_2)

timeR_output = Dense(1, kernel_initializer='glorot_uniform', name='timeR_output')(b2_3)

model = Model(inputs=[main_input], outputs=[act_output, time_output, timeR_output])

model.compile(loss={'act_output': 'categorical_crossentropy', 'time_output': 'mae', 'timeR_output': 'mae'}, optimizer='nadam')

early_stopping = EarlyStopping(monitor='val_loss', patience=50)
model_checkpoint = ModelCheckpoint('output_files/models/model_'+eventlog+'_{epoch:02d}-{val_loss:.2f}.h5', monitor='val_loss',
                                   verbose=0, save_best_only=True, save_weights_only=False, mode='auto')
lr_reducer = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=15, verbose=0, mode='auto', min_delta=0.0001,
                               cooldown=0, min_lr=0)

history = model.fit(X, {'act_output': y_a, 'time_output': y_t, 'timeR_output': y_tr}, validation_split=0.2, verbose=2,
          callbacks=[early_stopping, model_checkpoint, lr_reducer], batch_size=maxlen, epochs=500)
# list all data in history

In [ ]:
# list all data in history
print(history.history.keys())
# summarize history for accuracy
# plt.plot(history.history['accuracy'])
# plt.plot(history.history['val_accuracy'])
# plt.title('model accuracy')
# plt.ylabel('accuracy')
# plt.xlabel('epoch')
# plt.legend(['train', 'test'], loc='upper left')
# plt.show()
# # summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
#### comparing if product and item match but is to hard to handle them

t1 = [len(t) for t in ocel['Items']]
t2 = [len(t) for t in ocel['Products']]
t3 = [a!=b for a, b in zip(t1, t2)]
print(ocel.size)
print(sum(t3))
print(sum(t3)/ocel.size)
print(len(t3) - sum(t3))
# display(ocel[t3])
# display(ocel[t3].explode('Products').groupby(['Products']).size().sort_values())
oceltemp = ocel[t3].copy()
t1 = [len(t) for t in oceltemp['Items']]
t2 = [len(t) for t in oceltemp['Products']]
t3 = [a!=b for a, b in zip(t1, t2)]
print(sum(t3))
df_agg = ocel.copy()
df_agg.Items = df_agg.Items.apply(len)
df_agg.Orders = df_agg.Orders.apply(len)
df_agg.Products = df_agg.Products.apply(len)
df_agg = df_agg.groupby('Packages').sum(numeric_only=True)
display(df_agg)